<a href="https://colab.research.google.com/github/joelwwiggins/car_ml/blob/main/Copy_of_notebook_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
#import csv from path
#drop first row and column
df1 = pd.read_csv('CSVLog_20221130_091051.csv', skiprows=1)


In [3]:
#drop first column
df1_col=df1.iloc[:,1:]
df=df1_col

In [4]:
df.head()

,Latitude (deg),Longitude (deg),Vehicle speed (MPH),Instant fuel economy (MPG),Total fuel economy (MPG),Fuel rate (gal/hr),Instant CO2 rate (lb/mile),Total CO2 (lbs),CO2 flow (lb/min),Accel X (ft/s²),...,Average Trip CO2 Rate (lb/mile),Max Speed (MPH),Seconds Idling (sec),Idling Count,Hard Accel Count,Hard Brake Count,Trip Duration (min),Trip Fuel Economy (MPG),Trip Fuel (gal),Trip Distance (miles)
0,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
1,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.294891,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
2,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.317437,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
3,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.317437,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
4,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.346312,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0


In [15]:
#convert to numpy array
df_np=df.to_numpy()

#train test split
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df_np, test_size=0.2, random_state=42)

#normalize the x_train
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

#reshape the data to 3d
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1],1))

#use a 1dconv model to predict the next value
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D 
from keras.layers import Dropout
from keras.layers import LSTM

model_shape = X_train_scaled.shape



nn_model = tf.keras.models.Sequential()
nn_model.build(input_shape=model_shape)

nn_model.add(tf.keras.layers.Conv1D(512,3,activation="relu",padding='same',input_shape=model_shape[1:]))

nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))


nn_model.add(tf.keras.layers.Flatten())
nn_model.add(tf.keras.layers.Dense(units=model_shape[1], activation='linear'))




nn_model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
es = tf.keras.callbacks.EarlyStopping(monitor='val_mae',patience=10)
# checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath ="/content/sample_data")
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_images=True)
cb_list = [es, tbCallBack]
nn_model.fit(X_train, X_train, epochs=30,validation_split=.1,shuffle=True,callbacks=cb_list)
#normalize the x_test
X_test_scaled = scaler.fit_transform(X_test)
nn_model.evaluate(X_test_scaled, X_test_scaled)



nn_model.summary()

Epoch 1/30
845/845 [==============================] - 6s 6ms/step - loss: 233702.5312 - mae: 35.3989 - mse: 233702.5312 - val_loss: 1103.2548 - val_mae: 23.1273 - val_mse: 1103.2548
Epoch 2/30
845/845 [==============================] - 5s 6ms/step - loss: 1179.8273 - mae: 24.7761 - mse: 1179.8273 - val_loss: 7235.0308 - val_mae: 68.4246 - val_mse: 7235.0308
Epoch 3/30
845/845 [==============================] - 5s 6ms/step - loss: 1035.0907 - mae: 21.8103 - mse: 1035.0907 - val_loss: 1389.3247 - val_mae: 28.1402 - val_mse: 1389.3247
Epoch 4/30
845/845 [==============================] - 5s 6ms/step - loss: 825.4521 - mae: 20.4932 - mse: 825.4521 - val_loss: 607.1544 - val_mae: 19.5156 - val_mse: 607.1544
Epoch 5/30
845/845 [==============================] - 5s 6ms/step - loss: 617.9431 - mae: 18.2374 - mse: 617.9431 - val_loss: 343.6556 - val_mae: 13.9129 - val_mse: 343.6556
Epoch 6/30
845/845 [==============================] - 5s 6ms/step - loss: 494.0555 - mae: 15.7018 - mse: 494.0555 

In [ ]:


#plot the loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

#predict the next value
X_test_scaled = scaler.transform(X_test)
yhat = model.predict(X_test_scaled)

#inverse transform the predicted value
yhat_inv = scaler.inverse_transform(yhat)

#inverse transform the actual value
X_test_inv = scaler.inverse_transform(X_test)

#plot the actual and predicted value
plt.plot(X_test_inv[:,0], label='actual')
plt.plot(yhat_inv[:,0], label='predicted')
plt.legend()
plt.show()

